In [1]:
from apiclient.discovery import build
import argparse
import csv
import time
import pandas as pd
import dateutil.parser
import isodate
from datetime import datetime 
import pytz 


In [ ]:
DEVELOPER_KEY = #INSERT YOUR KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [ ]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
# Call the search.list method to retrieve results matching the specified
# query term
query = input("Enter search query: ")

IST = pytz.timezone('Asia/Kolkata')   
datetime_ist = datetime.now(IST) 
TimeQueried =  datetime_ist.strftime('%d/%m/%Y %H:%M:%S ')
#print(TimeQueried)


search_response = []
response_page = youtube.search().list(q=query, part="id,snippet", type="video", maxResults=100).execute()
search_response.append(response_page)
nextPageToken = response_page['nextPageToken']


for _ in range(4):
    response_page = youtube.search().list(q=query, part="id,snippet", pageToken=nextPageToken, type="video", maxResults=100).execute()
    search_response.append(response_page)
    if ("nextPageToken" in response_page):
        nextPageToken = response_page['nextPageToken']
        continue
    else:
        break

In [ ]:
search_response

In [ ]:
id_list = []
for v in search_response:
    for v_item in v['items']:
        id_list.append(v_item['id']['videoId'])

In [ ]:
video_response = []
for i in id_list:
    videos = youtube.videos().list(id=i, part="id,snippet,statistics,contentDetails").execute()
    print(videos)
    video_response.append(videos['items'][0])

In [ ]:
video_response

In [ ]:
v_list = []
for v in video_response:

    duration = v['contentDetails']['duration']  
    dur=isodate.parse_duration(duration)
    sec = dur.total_seconds()
    stats = v['statistics']
    viewCount = stats.get('viewCount')
    likeCount = stats.get('likeCount')
    dislikeCount = stats.get('dislikeCount')
    favoriteCount = stats.get('favoriteCount')
    commentCount = stats.get('commentCount')
    
    v_dict = {'Topic': query, 'Time_Queried': TimeQueried , 'Video_Title': v['snippet']['title'], 'videoId': v['id'], "Published_At": v['snippet']['publishedAt'], "duration(sec)": sec , "viewCount": viewCount , "likeCount": likeCount , "dislikeCount": dislikeCount, "favoriteCount": favoriteCount, "commentCount":commentCount}
    v_list.append(v_dict)
    #print(commentCount)

In [ ]:
import csv
header = ['Topic', 'Time_Queried', 'Video_Title', 'videoId', 'Published_At', 'duration(sec)', 'viewCount', 'likeCount', 'dislikeCount', 'favoriteCount','commentCount']
file = 'Data.csv'

try:
    with open(file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        for data in v_list:
            writer.writerow(data)
except IOError:
    print('IO Error')

In [ ]:
df = pd.read_csv("Data.csv", header=None)
print(df)

In [ ]:
####################### MERGING ######################

In [ ]:
import pandas as pd
from scipy import stats 
from sklearn import preprocessing
from math import sqrt

df1 = pd.read_csv (r'Day1.csv')
df2 = pd.read_csv (r'Day2.csv')
df3 = pd.read_csv (r'Day3.csv')
df4 = pd.read_csv (r'Day4.csv')
#print(df1)

In [ ]:
############ Ensuring that there are no 'nan' in the data frame which might cause a problem ############# 

df1["likeCount"] = df1["likeCount"].fillna(0)
df2["likeCount"] = df2["likeCount"].fillna(0)
df3["likeCount"] = df3["likeCount"].fillna(0)
df4["likeCount"] = df4["likeCount"].fillna(0)


df1["dislikeCount"] = df1["dislikeCount"].fillna(0)
df2["dislikeCount"] = df2["dislikeCount"].fillna(0)
df3["dislikeCount"] = df3["dislikeCount"].fillna(0)
df4["dislikeCount"] = df4["dislikeCount"].fillna(0)



#print(df1)

In [ ]:
M=pd.merge(df1, df4, how='inner',on=['videoId','Video_Title','PublishedAt','duration(sec)','Topic'])

M['viewCount_diff']=M['viewCount_y']-M['viewCount_x']
M['likeCount_diff']=M['likeCount_y']-M['likeCount_x']
M['dislikeCount_diff']=M['dislikeCount_y']-M['dislikeCount_x']
M['commentCount_diff']=M['commentCount_y']-M['commentCount_x']

#print(M)

In [ ]:
grouped = M.groupby('Topic')
Topic1 = grouped.get_group("DATA SCIENCE")
Topic2 = grouped.get_group("COMPUTER NETWORKS")
Topic3 = grouped.get_group("WIRELESS COMMUNICATION")
#print(Topic3)

In [ ]:
################ NORMALISING LIKE_DISLIKE_DIFF AND VIEWCOUNT ########################

Topic1['like_dislike_diff'] = Topic1['likeCount_diff'] -Topic1['dislikeCount_diff']
s = sum(Topic1['viewCount_diff'])
Topic1['viewCount_diff_Norm'] = Topic1['viewCount_diff']/s
s = sum(Topic1['like_dislike_diff'])
Topic1['like_dislike_diff_Norm'] = Topic1['like_dislike_diff']/s

#print(Topic1)

In [ ]:
Topic2['like_dislike_diff'] = Topic2['likeCount_diff'] -Topic2['dislikeCount_diff']
s = sum(Topic2['viewCount_diff'])
Topic2['viewCount_diff_Norm'] = Topic2['viewCount_diff']/s
s = sum(Topic2['like_dislike_diff'])
Topic2['like_dislike_diff_Norm'] = Topic2['like_dislike_diff']/s

#print(Topic2)

In [ ]:
Topic3['like_dislike_diff'] = Topic3['likeCount_diff'] -Topic3['dislikeCount_diff']
s = max(Topic3['viewCount_diff'])
Topic3['viewCount_diff_Norm'] = Topic3['viewCount_diff']/s
s = max(Topic3['like_dislike_diff'])
Topic3['like_dislike_diff_Norm'] = Topic3['like_dislike_diff']/s

# print(Topic3)

In [ ]:
########################### SCORING ###############################

In [ ]:
#weight for viewCount_diff_Norm 
w1= 11
#weight for like_dislike_diff_norm 
w2= 5

In [ ]:
def score(viewCount_diff_Norm ,like_dislike_diff_Norm ):
    scr = viewCount_diff_Norm * w1 +like_dislike_diff_Norm * w2
    return scr


In [ ]:
##FOR TOPIC1##
l1=len(Topic1)
Topic1['score']=0.0
for i in range (l1):
    Topic1['score'][i]= score(Topic1['viewCount_diff_Norm'][i],Topic1['like_dislike_diff_Norm'][i]) 
    #calling the score function and calculating the score

    
##FOR TOPIC2##
l2=len(Topic2)
Topic2['score']=0.0
for i in range (l2):
    i = i + l1
    Topic2['score'][i]= score(Topic2['viewCount_diff_Norm'][i],Topic2['like_dislike_diff_Norm'][i])
    #calling the score function and calculating the score


##FOR TOPIC3##
l3=len(Topic3)
Topic3['score']=0.0
for i in range (l3):
    i = i + l1+ l2
    Topic3['score'][i]= score(Topic3['viewCount_diff_Norm'][i],Topic3['like_dislike_diff_Norm'][i])
    #calling the score function and calculating the score



In [ ]:
Topic1 = Topic1.sort_values("score",ascending = False) #sorting them in descending order on the basis of their scores
Topic2 = Topic2.sort_values("score",ascending = False) #sorting them in descending order on the basis of their scores
Topic3 = Topic3.sort_values("score",ascending = False) #sorting them in descending order on the basis of their scores
print(Topic3)

In [ ]:
##################### MAKING PLAYLIST FOR TOPIC1 ####################

In [ ]:
dur=0
num = 0
videotitle1=[]
duration1=[]
for i in Topic1.index:
    if (dur + Topic1['duration(sec)'][i] <= 54000):
            dur = dur + Topic1['duration(sec)'][i]
            num = num +1
            videotitle1.append(Topic1['Video_Title'][i])
            duration1.append(Topic1['duration(sec)'][i])
            print(num)
            print("TOPIC --> " + Topic1['Topic'][i]+" ; "+"VIDEO_TITLE --> " + Topic1['Video_Title'][i]+" ; "+
                  "VIDEO_ID --> " + Topic1['videoId'][i]+" ; "+ "PUBLISHED_AT --> "+ Topic1['PublishedAt'][i])
            print("DURATION_IN_SEC --> "+ str(Topic1['duration(sec)'][i]))
            print('\n')

In [ ]:
###################### MAKING PLAYLIST FOR TOPIC2 #####################

In [ ]:
dur=0
num = 0
videotitle2=[]
duration2=[]
for i in Topic2.index:
    if (dur + Topic2['duration(sec)'][i] <= 54000):
            dur = dur + Topic2['duration(sec)'][i]
            num = num +1
            videotitle2.append(Topic2['Video_Title'][i])
            duration2.append(Topic2['duration(sec)'][i])
            print(num)
            print("TOPIC --> " + Topic2['Topic'][i]+" ; "+"VIDEO_TITLE --> " + Topic2['Video_Title'][i]+" ; "+
                  "VIDEO_ID --> " + Topic2['videoId'][i]+" ; "+ "PUBLISHED_AT --> "+ Topic2['PublishedAt'][i])
            print("DURATION_IN_SEC --> "+ str(Topic2['duration(sec)'][i]))
            print('\n')

In [ ]:
###################### MAKING PLAYLIST FOR TOPIC3 ############################

In [ ]:
dur=0
num = 0
#videoid = []
videotitle3=[]
duration3=[]
# likecount=[]
for i in Topic3.index:
    if (dur + Topic3['duration(sec)'][i] <= 54000):
            dur = dur + Topic3['duration(sec)'][i]
            num = num +1
            #videoid.append(Topic3['videoId'][i])
            #likecount.append(Topic3['likeCount_diff'])
            videotitle3.append(Topic3['Video_Title'][i])
            duration3.append(Topic3['duration(sec)'][i])
            print(num)
            print("TOPIC --> " + Topic3['Topic'][i]+" ; "+"VIDEO_TITLE --> " + Topic3['Video_Title'][i]+" ; "+
                  "VIDEO_ID --> " + Topic3['videoId'][i]+" ; "+ "PUBLISHED_AT --> "+ Topic3['PublishedAt'][i])
            print("DURATION_IN_SEC --> "+ str(Topic3['duration(sec)'][i]))
            print('\n')

In [ ]:
##################### VISUALIZATION ###########

In [ ]:
import random
import numpy
import matplotlib.pyplot as plt 


In [ ]:
#GRAPH BETWEEN THE  TITLE'S OF PLAYLIST OF TOPIC2 AND THEIR DURATION
x=videotitle2
y=duration2
plt.plot(x,y)
plt.xlabel("videoTitle")
plt.ylabel("Duration(sec)")
plt.title("Topic2_videotitle vs duration(sec)")
pyplot.show()

In [ ]:
#GRAPH BETWEEN THE  TITLE'S OF PLAYLIST OF TOPIC3 AND THEIR DURATION
x=videotitle3
y=duration3
plt.plot(x,y)
plt.xlabel("videoTitle")
plt.ylabel("Duration(sec)")
plt.title("Topic3_videotitle vs duration(sec)")

pyplot.show()